In [1]:
%%capture 
import csv
from torch.utils.data import Dataset
import torch
from sklearn.model_selection import train_test_split
import numpy as np
from bs4 import BeautifulSoup
import string
import spacy
import jsonlines
import json
import re
import torch.nn as nn
from torch.nn.utils.rnn import pad_packed_sequence,pack_padded_sequence,pad_sequence
import torch.nn.functional as F
import torch.optim as optim
import fasttext
from torch.utils.data import SubsetRandomSampler,DataLoader,Subset
from torchtext.vocab import GloVe
from tqdm import tqdm
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

EMBED_DIM = 300
HIDDEN_DIM = 128

USE_TRAINABLE_EMBEDDINGS = True

PATIENCE_PARAMETER = 4
VALIDATION_LOSS_COMPUTE_STEP = 1


FILTER_SIZES = [3,4,5,6,7]
NUM_FILTERS = 50

device_cpu = torch.device('cpu')
device_fast = torch.device('cpu')

if torch.has_mps:
    device_fast = torch.device('mps')
elif torch.has_cuda:
    device_fast = torch.device('cuda')

#torch.manual_seed(0)
#np.random.seed(0)
nlp = spacy.load('en_core_web_sm')
glove = GloVe()

In [2]:
from torchtext.vocab import vocab

unk_index = 2196017
glove_vocab = vocab(glove.stoi)
pretrained_embeddings = glove.vectors
pretrained_embeddings = torch.cat((pretrained_embeddings,torch.zeros(1,pretrained_embeddings.shape[1]),))

In [3]:

def preprocess_text(text):
        
    text = re.sub(r'<br /><br />',".",text)
    text = BeautifulSoup(text,'lxml').get_text().strip()
    text = text.lower()

    #text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r"http\S+", "", text)
    text = ' '.join(re.findall(r"[\w']+|[.,!;/\"]", text))
    
    new_text = []
    for word in text.split():
        if word == '':
            continue
        new_text.append(word)
        #if '.' in word and len(word)>1:
            #if 'http' in word:
            #    words = word.split('.')
            #    words[1] = ''.join(words[1:])
            #    words = [words[0],words[1]]
            #else:
            #    words = word.split('.')
           
        #    words = word.split('.')
        #    for w in words:
        #        new_text.append(w)
        #else:
    
    text = ' '.join(new_text)
    words = nlp(text)
    text =  " ".join([token.text for token in words if not token.is_punct or token.text=='/' or token.text=="\"" or token.text=="."]).strip()
    #review =  " ".join([re.sub(r'^https?:\/\/.*[\r\n]*', '', token.text, flags=re.MULTILINE) for token in words])
    #review  = " ".join(new_text)
    new_words = []
    for word in text.split(" "):
        
        #vocab_words.add(word)
        if word == 'n\'t':
            if len(new_words) > 1:
                new_words[-1] = new_words[-1] + word
            else:
                new_words.append(word)
        else:
            new_words.append(word)
    text = " ".join(new_words)
    return text


In [4]:
preprocessed_dataset = []
train_dataset_labels = []
with open("./Train dataset.csv") as csvfile:
    csvFile = csv.reader(csvfile)
    next(csvFile)
    json_writer = jsonlines.open('processed_dataset.jsonl','w')

    for line in csvFile:
        processed_text = preprocess_text(line[0])
        label = 1.0 if line[1] == 'positive' else 0.0
        train_dataset_labels.append(label)
        json_writer.write({"text":processed_text,"label":label})
        preprocessed_dataset.append({"text":processed_text,"label":label})
    
    json_writer.close()

train_dataset_labels = np.array(train_dataset_labels)
#final_words.write(str(vocab_words))
#final_words.close()
#file.close()
#punctuation_words.close()


/Users/depressedcoder/environments/gymenv/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [5]:
preprocessed_dataset = []
train_dataset_labels = []
with open('processed_dataset.jsonl') as f:
    for line in f:
        sample = json.loads(line)
        train_dataset_labels.append(sample['label'])
        preprocessed_dataset.append(sample)
train_dataset_labels = np.array(train_dataset_labels)


In [8]:
def getWordEmbeddingforText(text):
    
    length = 0
    words = []
    for word in text.split(' '):
        length+=1
        if word in glove_vocab:
            word_index = glove_vocab[word]
        else:
            word_index = unk_index
        words.append(word_index)

    if length < max(FILTER_SIZES):
        while length < max(FILTER_SIZES):
            length+=1
            words.append(unk_index)

    return torch.tensor(words),length

In [9]:
processed_dataset = []

for review in preprocessed_dataset:
    embedding,length = getWordEmbeddingforText(review['text'])
    processed_dataset.append({'text': embedding,'length': length,'label' : review['label']})
 

In [10]:
class ReviewDataSet(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

dataset = ReviewDataSet(processed_dataset)

In [16]:
# Train and Validation split and an equal distriubition of classes
train_idx,valid_idx = train_test_split(np.arange(train_dataset_labels.shape[0]), 
    test_size=0.2,
    shuffle= True,
    stratify= train_dataset_labels,
    random_state=0
)

def collate_function(batch_data):
    inputs = [b['text'] for b in batch_data]
    lengths = [b['length'] for b in batch_data]
    labels = torch.tensor([b['label'] for b in batch_data])

    labels = labels.unsqueeze(1)
    inputs = pad_sequence(inputs,batch_first=True,padding_value=unk_index)
    return  {'input' : inputs , 'lengths': lengths , 'labels' : labels }

    

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
train_dataloader = DataLoader(dataset,64,sampler=train_sampler,collate_fn=collate_function)
valid_dataloader = DataLoader(dataset,64,sampler=valid_sampler,collate_fn=collate_function)

In [18]:
data = next(iter(train_dataloader))

In [20]:
data['input'].shape

torch.Size([64, 730])

In [38]:
class EmbeddingLayer(nn.Module):

    def __init__(self,embeddings):
        super().__init__()
        self.embeddings_layer = nn.Embedding.from_pretrained(pretrained_embeddings, freeze = False)
    def forward(self,x):
        return self.embeddings_layer(x)


In [39]:
class DAN(nn.Module):
    
    def __init__(self,embed_dim=EMBED_DIM,hidden_dim = HIDDEN_DIM, droput_prob = 0.3, train_device = device_cpu):
        super().__init__()
        
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(self.embed_dim,self.hidden_dim)
        #self.fc = nn.Linear(self.hidden_dim,self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim,1)
        self.sigmoid  = nn.Sigmoid()
        self.word_dropout_prob = droput_prob

        self.train_device = train_device
        self.embedding_layer = EmbeddingLayer(pretrained_embeddings)

    def forward(self,inp,inp_len):    
        inp = self.embedding_layer(inp)
        inp_mask = torch.ones((inp.shape[0],inp.shape[1]))
        for i  in range(inp.shape[0]):
            inp_mask[i,inp_len[i]:] = 0.0

        inp_mask = inp_mask.to(self.train_device)
        inp_lengths = torch.sum(inp_mask,-1,keepdim=True).int()
        inp_lengths = inp_lengths.to(self.train_device)
        total = torch.sum(inp*(inp_mask.unsqueeze(2)),axis=1)
        vector_average = total / inp_lengths
        ans = F.relu(self.fc1(vector_average))
        #ans = F.relu(self.fc(ans))
        ans = self.sigmoid(self.fc2(ans))
        return ans

dan = DAN()


In [ ]:
class RNNModel(nn.Module):

    def __init__(self,
            embed_dim=EMBED_DIM,hidden_dim =HIDDEN_DIM,bidirectional=False,
            rnn_type = 'gru',num_layers=1,rnn_dropout = 0.4,fc_dropout = 0.3):
        
        super().__init__()
        self.embedding_layer = EmbeddingLayer(pretrained_embeddings)

        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.dropout = rnn_dropout
        self.fc_dropout = nn.Dropout(fc_dropout)

        self.rnn = None
        
        if rnn_type == 'gru':
            self.rnn  = nn.GRU(input_size = self.embed_dim,
                hidden_size = self.hidden_dim,
                num_layers = self.num_layers,
                batch_first = True,
                dropout = self.dropout,
                bidirectional = self.bidirectional
            )
        elif rnn_type == 'rnn':
            self.rnn = nn.RNN(input_size = self.embed_dim,
                hidden_size = self.hidden_dim,
                num_layers = self.num_layers,
                batch_first = True,
                dropout = self.dropout,
                bidirectional = self.bidirectional)
        

        self.bidirectional_factor = 2 if self.bidirectional else 1
        self.dnn_input_size= self.bidirectional_factor * self.num_layers * self.hidden_dim
        self.fc_list = []

        log_base_2 = np.log2(self.dnn_input_size)
        nearest_power_2 = int(log_base_2)

        if(float(nearest_power_2) != log_base_2):
            self.fc_list.append(nn.Linear(self.dnn_input_size,2**nearest_power_2))

        while nearest_power_2 > 7 :
            self.fc_list.append(nn.Linear(2**(nearest_power_2),2**(nearest_power_2-1)))
            nearest_power_2-=1
        
        self.fc_list.append(nn.Linear(128,1))
        self.fc = nn.ModuleList(self.fc_list)

        #self.fc1 = nn.Linear(self.dnn_input_size,128)
        #self.fc2 = nn.Linear(128,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x,x_len):
        x = self.embedding_layer(x)
        packed_input = pack_padded_sequence(x,x_len,batch_first=True,enforce_sorted=False)
        packed_output,hidden = self.rnn(packed_input)
        output,output_lengths = pad_packed_sequence(packed_output)
        #hidden = hidden.squeeze()
        hidden = torch.permute(hidden,(1,0,2))
        hidden = hidden.contiguous().view((hidden.shape[0],-1))
        out = hidden
        for i,l in enumerate(self.fc):
            if i!=(len(self.fc_list)-1):
                out = self.fc_dropout(l(out))
            
        #out = self.fc_dropout(self.fc1(hidden))
        ans = self.sigmoid(self.fc_list[len(self.fc_list)-1](out))
        return ans



In [ ]:
sent = RNNModel(bidirectional=True,num_layers=1)

In [ ]:
# ADD Dropout Term

class CNNModel(nn.Module):

    def __init__(self, embed_dim=EMBED_DIM,hidden_dim = HIDDEN_DIM,filter_sizes = FILTER_SIZES, n_filters = NUM_FILTERS,dropout = 0.2):
        super().__init__()
        self.embedding_layer = EmbeddingLayer(pretrained_embeddings)
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.filter_sizes = filter_sizes
        self.num_filters = n_filters
        self.dropout = nn.Dropout(p=dropout)
        
        self.modulelist = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
            out_channels=self.num_filters,
            kernel_size= self.filter_sizes[i]
            )
    
        for i in range(len(self.filter_sizes))])
        
        self.fc1 = nn.Linear(self.num_filters*len(self.filter_sizes),hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x : torch.Tensor,xlen = None):
        x = self.embedding_layer(x)
        permuted_x = x.permute(0,2,1)
        x_conv_list = [F.relu(conv(permuted_x)) for conv in self.modulelist]
        x_max_pool_list = [F.max_pool1d(x_conv,kernel_size=x_conv.shape[2]) for x_conv in x_conv_list]
        
        feature = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_max_pool_list],dim=1)
        
        out = self.dropout(F.relu(self.fc1(feature)))
        out = self.fc2(out)
        return self.sigmoid(out)



cnnmodel = CNNModel(10,2,FILTER_SIZES,NUM_FILTERS)    

In [ ]:
def length_after_1d_convolution(x,filter_size,max_pool_size):

    def fn(sent_len,a=filter_size,pool_size = max_pool_size):
        conv_out = sent_len - a +1
        return int(conv_out/pool_size)

    ans = list(map(fn,x))
    return ans

class EnsembleModel(nn.Module):

    def __init__(self,embed_dim = EMBED_DIM,hidden_dim = HIDDEN_DIM,filter_size = 3,num_filters=256,max_pool_size = 3,dropout = 0.3,
        rnn_bidirectional = True,
        rnn_num_layers = 1,
        rnn_dropout = 0.0
    ):
        super().__init__()
        self.embedding_layer = EmbeddingLayer(pretrained_embeddings)
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.filter_size = filter_size
        self.max_pool_size = max_pool_size
        self.dropout = dropout

        self.conv = nn.Conv1d(self.embed_dim,num_filters,self.filter_size)
        self.pool = nn.MaxPool1d(max_pool_size)

        self.RNNModel = RNNModel(num_filters,hidden_dim,rnn_bidirectional,num_layers=rnn_num_layers,rnn_dropout=rnn_dropout,fc_dropout=dropout)


    def forward(self,x,x_len):
        x = self.embedding_layer(x)
        x = x.permute(0,2,1)
        conv_x = F.relu(self.conv(x))
        pooled_x = self.pool(conv_x)

        new_x = pooled_x.permute(0,2,1)
        #print(new_x.shape)
        new_x_len = length_after_1d_convolution(x_len,self.filter_size,self.max_pool_size)
        #print(new_x_len)

        ans = self.RNNModel(new_x,new_x_len)
        return ans
ensemble_model = EnsembleModel()


In [40]:
import os
from torch.utils.tensorboard import SummaryWriter
from datetime import  datetime

def train(model,train_dataloader,valid_dataloader,num_epochs,criterion,optimizer,check_point_name,tensorboard_name,device_train = device_fast,use_rnn = False,log=True):
    
    if log == True:
        current_datetime = datetime.now().strftime("%d/%m/%Y_%H:%M:%S")
        tensorboard_name = tensorboard_name + "_" + current_datetime
        writer = SummaryWriter('runs/' + tensorboard_name)
    
    model = model.to(device_train)


    clip = 0
    if use_rnn:
        clip = 5

    best_validation_loss = 1000.0
    valdiation_loss_not_decreased_steps = 0
    
    model.train()
    for e in range(num_epochs):
        
        training_set_size = 0
        training_loss = 0.0
        model.train()

        for data in tqdm(train_dataloader):
            
            optimizer.zero_grad()
            input_reviews,inp_lengths,output_labels = data['input'], data['lengths'],data['labels']
            input_reviews = input_reviews.to(device_train)
            training_set_size += input_reviews.shape[0]
            output = model(input_reviews,inp_lengths)
            output = output.to(device_cpu)
            loss = criterion(output,output_labels.float())
            training_loss += loss.item()
            loss.backward()
            if use_rnn:
                nn.utils.clip_grad_norm_(model.parameters(),clip)
            optimizer.step()
        
        current_training_loss = training_loss / training_set_size
        if log==True:
            print("Epoch " + str(e) + " Average Training Loss = " +  str(current_training_loss))
            writer.add_scalars(tensorboard_name + '; Loss vs Epoch',{'train' : current_training_loss},e)
        
        model.eval()
        
        if valid_dataloader is None:
            continue
        
        validation_set_size  = 0 
        if e% VALIDATION_LOSS_COMPUTE_STEP==0:
            correct_count = 0
            validation_loss = 0

            for i,data in enumerate(valid_dataloader,0):
                input_reviews,inp_lengths,output_labels = data['input'], data['lengths'],data['labels']
                input_reviews = input_reviews.to(device_train)
                validation_set_size += input_reviews.shape[0]
                output = model(input_reviews,inp_lengths)
                output = output.to(device_cpu)
                loss = criterion(output,output_labels.float())
                validation_loss += loss.item()
                nearest_class = torch.round(output)

                correct = (nearest_class == output_labels.float()).float()
                correct_count += correct.sum()
            correct_count = int(correct_count)
            current_validation_accuracy = (correct_count/validation_set_size)*100
            current_validation_loss = (1.0* validation_loss)/validation_set_size
            if log == True:
                print("Epoch " + str(e) + " " +  "Validation Loss = " + str(current_validation_loss) )
                print("Validation Set Accuracy = " + str((correct_count/validation_set_size)*100) )

                writer.add_scalar(tensorboard_name + ' Validation Accuracy vs Epoch ',int((correct_count/validation_set_size)*100),e)
                writer.add_scalars('Loss vs Epoch',{'valid' : current_validation_loss},e)
            
            if log==True:
                if current_validation_loss < best_validation_loss:
                    valdiation_loss_not_decreased_steps = 0
                    torch.save(model.state_dict(),check_point_name)
                    best_validation_loss = current_validation_loss
                else:
                    valdiation_loss_not_decreased_steps +=1
            else:   
                with tune.checkpoint_dir(e) as checkpoint_dir:
                    g = check_point_name[14:].split('_')
                    checkpoint_name = 'tune_checkpoints/' + '_'.join(g)
                    path = os.path.join(checkpoint_dir,check_point_name)
                    torch.save((model.state_dict(),optimizer.state_dict()),path)
                tune.report(loss = current_validation_loss,accuracy = current_validation_accuracy)   

        if log == True:
            if valdiation_loss_not_decreased_steps >= PATIENCE_PARAMETER:
                break

In [44]:
EPOCHS = 50
dan = DAN(train_device=device_fast)
criterion = nn.BCELoss()
LR = 0.01
optimizer = optim.Adam(dan.parameters(),lr=LR)

model_name = type(dan).__name__
optimizer_name = type(optimizer).__name__
loss_fn_name = type(criterion).__name__

checkpoint_name = 'checkpoints/'+model_name+ '_' + str(EPOCHS)+"_"+ loss_fn_name  +"_"+ optimizer_name + "_" + str(LR)+'_.pth'  
train(dan,train_dataloader,valid_dataloader,EPOCHS,criterion,optimizer,checkpoint_name,model_name,device_fast,use_rnn=False)

  0%|          | 1/500 [00:30<4:11:36, 30.25s/it]


KeyboardInterrupt: 

In [ ]:
def start_training(params,train_dataloader,valid_dataloader):


    def get_optimizer(model : nn.Module, optim_fn_name,lr):

        if optim_fn_name=='adam':
            return optim.Adam(model.parameters(),lr=lr)
        elif optim_fn_name == 'adagrad':
            return optim.Adagrad(model.parameters(),lr=lr)
        else:
            return optim.SGD(model.parameters(),lr=lr,nesterov=True)
    
    checkpoint_name = ''
    model_type = params['model']
    epochs =  params['epochs']
    loss_function = params['criterion']
    optimizer_function = params['optimizer']
    learning_rate = params['lr']
    
  
    criterion = nn.BCELoss()
    if loss_function == 'bce':
        criterion = nn.BCELoss()

    optimizer = None
    model_name = ''
   
    model = None
    # Deep Averaging Network
    if model_type == 1:
        model = DAN()
        optimizer = get_optimizer(model,optimizer_function,learning_rate)
        model_name = type(model).__name__
        loss_fn_name = type(criterion).__name__
        optimizer_name = type(optimizer).__name__
        checkpoint_name = 'checkpoints/'+model_name+ '_' + str(epochs)+"_"+ loss_fn_name  +"_"+ optimizer_name + "_" + str(learning_rate)+'_.pth'  

    # RNN Model
    elif model_type==2:
        bidirectional = params['bidirectional']
        bidir = 'T' if bidirectional else 'F'
        num_layers  = params['num_layers']
        rnn_type = params['rnn_type']
        rnn_dropout = 0.3 if num_layers > 1 else 0.0
        fc_dropout =  params['fc_dropout'] if 'fc_dropout' in params else 0.3 
        model = RNNModel(rnn_type=rnn_type,bidirectional=bidirectional,num_layers=num_layers,rnn_dropout=rnn_dropout,fc_dropout=fc_dropout)
        
        optimizer = get_optimizer(model,optimizer_function,learning_rate)
        model_name = type(model).__name__
        loss_fn_name = type(criterion).__name__
        optimizer_name = type(optimizer).__name__
        
        checkpoint_name = 'checkpoints/'+model_name+ '_' +rnn_type + '_'+ str(epochs)+"_"+ loss_fn_name  +"_"+ optimizer_name + "_" + str(learning_rate) + "_" + str(num_layers) +'_'+ bidir +'_' + str(rnn_dropout)+'_.pth' 

    # CNN Based Model
    elif model_type == 3:
        fc_dropout = params['fc_dropout'] if 'fc_dropout' in params else 0.2
        model = CNNModel(dropout = fc_dropout )
        optimizer = get_optimizer(model,optimizer_function,learning_rate)
        model_name = type(model).__name__
        loss_fn_name = type(criterion).__name__
        optimizer_name = type(optimizer).__name__
        checkpoint_name = 'checkpoints/'+model_name+ '_' + str(epochs)+"_"+ loss_fn_name  +"_"+ optimizer_name + "_" + str(learning_rate) +'_.pth' 
    # Ensemble Model
    else:
       
        bidirectional = params['bidirectional']
        bidir = 'T' if bidirectional else 'F'
        num_layers  = params['num_layers']
        rnn_dropout = 0.3 if num_layers > 1 else 0.0
        fc_dropout =  params['fc_dropout'] if 'fc_dropout' in params else 0.3 
        model = EnsembleModel(rnn_bidirectional=bidirectional,rnn_num_layers=num_layers,rnn_dropout=rnn_dropout,dropout=fc_dropout)
        model_name = type(model).__name__
        loss_fn_name = type(criterion).__name__
        optimizer_name = type(optimizer).__name__

        checkpoint_name = 'checkpoints/' + model_name +'_' + str(epochs)+'_'+ loss_fn_name  +'_'+ optimizer_name + '_' + str(learning_rate) + '_' + str(num_layers)+ '_' + bidir +'_' + str(rnn_dropout) +'_.pth' 

    train(model,train_dataloader,valid_dataloader,epochs,criterion,optimizer,checkpoint_name,model_name,params['device'],params['use_rnn'],params['log'])




In [ ]:
params = {
    'model' : 1,
    'epochs' : 50,
    'criterion' : 'bce',
    'optimizer' : 'adam',
    'lr' : 0.01,
    'bidirectional' : False,
    'num_layers' : 1,
    'rnn_type' : 'gru',
    'fc_dropout' : 0.3,
    'device' : device_cpu,
    'use_rnn' : False,
    'log' : True
}
start_training(params=params,train_dataloader=train_dataloader,valid_dataloader=valid_dataloader)

In [ ]:
EPOCHS = 50
BIDIRECTIONAL = 'F'
NUM_LAYERS = 1
bidirectional = True if BIDIRECTIONAL=='T' else False
RNN_TYPE = 'gru'
RNN_DROPOUT = 0.3 if NUM_LAYERS == 1 else 0.0

model = RNNModel(rnn_type=RNN_TYPE,bidirectional=bidirectional,num_layers=NUM_LAYERS,rnn_dropout=RNN_DROPOUT)
criterion = nn.BCELoss()
LR = 0.001
optimizer = optim.Adam(dan.parameters(),lr=LR)

model_name = type(model).__name__
optimizer_name = type(optimizer).__name__
loss_fn_name = type(criterion).__name__

checkpoint_name = 'checkpoints/'+model_name+ '_' +RNN_TYPE + '_'+ str(EPOCHS)+"_"+ loss_fn_name  +"_"+ optimizer_name + "_" + str(LR) + "_" + str(NUM_LAYERS) +'_'+ BIDIRECTIONAL +'_' + str(RNN_DROPOUT)+'_.pth' 
train(model,train_dataloader,valid_dataloader,EPOCHS,criterion,optimizer,checkpoint_name,model_name,device_cpu,use_rnn=False)

In [ ]:
EPOCHS = 50
model = CNNModel()
criterion = nn.BCELoss()
LR = 0.01
optimizer = optim.Adam(dan.parameters(),lr=LR)

model_name = type(model).__name__
optimizer_name = type(optimizer).__name__
loss_fn_name = type(criterion).__name__

checkpoint_name = 'checkpoints/'+model_name+ '_' + str(EPOCHS)+"_"+ loss_fn_name  +"_"+ optimizer_name + "_" + str(LR) +'_.pth' 
train(model,train_dataloader,valid_dataloader,EPOCHS,criterion,optimizer,checkpoint_name,model_name,device_cpu,use_rnn=False)

In [ ]:
EPOCHS = 50
BIDIRECTIONAL = 'T'
NUM_LAYERS = 1
bidirectional = True if BIDIRECTIONAL=='T' else False
RNN_DROPOUT = 0.3 if NUM_LAYERS == 1 else 0.0


model = EnsembleModel(rnn_bidirectional=bidirectional,rnn_num_layers=NUM_LAYERS,rnn_dropout=RNN_DROPOUT)
criterion = nn.BCELoss()
LR = 0.001
optimizer = optim.Adam(dan.parameters(),lr=LR)

model_name = type(model).__name__
optimizer_name = type(optimizer).__name__
loss_fn_name = type(criterion).__name__

checkpoint_name = 'checkpoints/' + model_name +'_' + str(EPOCHS)+'_'+ loss_fn_name  +'_'+ optimizer_name + '_' + str(LR) + '_' + str(NUM_LAYERS)+ '_' + BIDIRECTIONAL +'_' + str(RNN_DROPOUT) +'_.pth' 
train(model,train_dataloader,valid_dataloader,EPOCHS,criterion,optimizer,checkpoint_name,model_name,device_cpu,use_rnn=False)

### HyperParameter Search

In [ ]:

from functools import partial



def tune_RNNModel(config):
    rnnmodel = RNNModel( bidirectional=config['bidirectional'],num_layers=config['num_layers'],rnn_dropout= config['rnn_dropout'],
        fc_dropout= config['fc_dropout'])
    criterion = nn.BCELoss()
    optimizer = optim.Adam(rnnmodel.parameters(),lr=config['lr'])

    train(rnnmodel,train_dataloader,valid_dataloader,10,criterion,optimizer,'','',device_cpu,True,False)

def tune_CNNModel(config):
    pass

def tune_EnsembleModel(config):
    pass



# 
# Model Number 1 = RNN model, 2 = CNNModel , 3 = EnsembleModel
#

def tune_model(modelnumber,num_samples = 10, max_epochs = 10, gpus_per_trial = 0.0):
    

    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_epochs,
        grace_period=1,
        reduction_factor=2)
    
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])

    config = {}
    if tune_model == 1:
        config = {

            'bidirectional' : tune.choice([True,False]),
            'num_layers' : tune.choice([1,2,3]), 
            'rnn_dropout' :tune.choice([0.0]),
            'fc_dropout' : tune.uniform(0.2, 1.0),
            'lr' : tune.qloguniform(1e-4, 1e-1, 5e-5),

        }


        result = tune.run(
        tune_RNNModel,
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    elif tune_model == 2:
        pass
        result = tune.run(
        tune_RNNModel,
        #partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)
    else:
        pass  
        result = tune.run(
        tune_RNNModel,
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)
   

    
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))




In [ ]:
tune_model(1)

### Test phase

In [ ]:
def test(checkpoint_name,test_data,test_lengths,test_labels):
    model_name = checkpoint_name[14:]
    parameters =  model_name.split('_')
    count = 0
    model = None
    if parameters[0]=='DAN':
        model = DAN()
    elif parameters[0] == 'CNNModel':
        model = CNNModel()
    elif parameters[0] == 'RNNModel':
        bidir = True if parameters[7]=='T' else False
        model = RNNModel(num_layers=int(parameters[6]),rnn_type=parameters[1],bidirectional=bidir)
    elif parameters[0] == 'EnsembleModel':
        bidir = True if parameters[7] =='T' else False
        num_layers = int(parameters[6])
        model = EnsembleModel(rnn_bidirectional=bidir,rnn_num_layers=num_layers,rnn_dropout=float(parameters[8]))
    
    model.load_state_dict(torch.load(checkpoint_name))
    model.eval()
    for i in range(len(test_data)):
        ans = model(test_data[i],[test_lengths[i]])
        ans = torch.round(ans)
        if ans[0][0] == test_labels[i]:
            count+=1
    
    print("Accuracy = " + str(count/len(test_data)))


In [ ]:
test_dataset_labels = []  
test_processed_text = []
with open("./Train dataset.csv") as csvfile:
    csvFile = csv.reader(csvfile)
    next(csvFile)
    for line in csvFile:
        processed_text = preprocess_text(line[0])
        label = 1.0 if line[1] == 'positive' else 0.0
        test_dataset_labels.append(label)
        test_processed_text.append(processed_text)


test_word_embeddings = []
test_sentence_lengths = []
for text in test_processed_text:
    embeddings,length = getWordEmbeddingforText(text)
    test_sentence_lengths.append(length)
    test_word_embeddings.append(embeddings.unsqueeze(0))



In [ ]:
test('./checkpoints/dan_50_bce_adam_0.01',test_word_embeddings,test_sentence_lengths,test_dataset_labels)    

In [ ]:
embeddings,length = getWordEmbeddingforText("the movie is not that good.")
dan = DAN()
dan.load_state_dict(torch.load('./checkpoints/DAN_50_BCELoss_Adam_0.01_.pth'))
dan.eval()
dan(embeddings.unsqueeze(0),[length])